In [32]:
from pyneuroml import pynml
import urllib.request, json 
#import requests
import os
import numpy as np
from neuroml import *
from neuroml.utils import component_factory, validate_neuroml2
from pyneuroml import pynml
#from pyneuroml.lems import LEMSSimulation
from pyneuroml.lems import generate_lems_file_for_neuroml
import neuroml.writers as writers
import random
from pyneuroml.pynml import read_lems_file, read_neuroml2_file, write_lems_file, write_neuroml2_file


In [20]:
'''https://pyneuroml.readthedocs.io/en/development/pyneuroml.io.html
pyneuroml.io.read_lems_file
pyneuroml.io.read_neuroml2_file
pyneuroml.io.write_lems_file
pyneuroml.io.write_neuroml2_file'''

# use the above functions to read LEMS and NeuroML files
cell_data_dir = "../NMC_model/NMC.NeuronML2/"
network_data_dir = "../NMC_model/"  # contains JSONs for network and connectivity data

files = os.listdir(cell_data_dir)
cell_files = [f for f in files if f.endswith("cell.nml")]  # .cell.nml -- contains arborization location details
net_files = [f for f in files if f.endswith("net.nml")]  # .net.nml -- contains location (0,0,0) in x-y-z space
current_files = [f for f in files if (f.endswith(".nml") and 'current' in f)]  # .current_clamp.nml -- contains current clamp details
print(cell_files[:5])
print(net_files[:5])
print(current_files[:5])

# validate the NeuroML files
'''for file in cell_files:
    print("Validating: " + file)
    validate_neuroml2(cell_data_dir + file)
for file in net_files:
    validate_neuroml2(cell_data_dir + file)
for file in current_files:
    validate_neuroml2(cell_data_dir + file)
'''

['bAC217_L23_BP_2004171a4c_0_0.cell.nml', 'bAC217_L23_BP_2f6bdee09f_0_0.cell.nml', 'bAC217_L23_BP_4652c9a7e1_0_0.cell.nml', 'bAC217_L23_BP_472b717f1f_0_0.cell.nml', 'bAC217_L23_BP_ca85cefc69_0_0.cell.nml']
['bAC217_L23_BP_2004171a4c.net.nml', 'bAC217_L23_BP_2004171a4c.stepcurrent3.net.nml', 'bAC217_L23_BP_2f6bdee09f.net.nml', 'bAC217_L23_BP_2f6bdee09f.stepcurrent3.net.nml', 'bAC217_L23_BP_4652c9a7e1.net.nml']
['bAC217_L23_BP_2004171a4c.stepcurrent3.net.nml', 'bAC217_L23_BP_2f6bdee09f.stepcurrent3.net.nml', 'bAC217_L23_BP_4652c9a7e1.stepcurrent3.net.nml', 'bAC217_L23_BP_472b717f1f.stepcurrent3.net.nml', 'bAC217_L23_BP_ca85cefc69.stepcurrent3.net.nml']


'for file in cell_files:\n    print("Validating: " + file)\n    validate_neuroml2(cell_data_dir + file)\nfor file in net_files:\n    validate_neuroml2(cell_data_dir + file)\nfor file in current_files:\n    validate_neuroml2(cell_data_dir + file)\n'

In [ ]:
# read in the network and connection properties from jsons: circuit.json, layers.json
with open(network_data_dir + 'circuit.json') as f:
    circuit_data = json.load(f)  # layer thickness and neuron counts
with open(network_data_dir + 'layers.json') as f:
    layers_data = json.load(f)  # layer-specific me-type counts
with open(network_data_dir + 'pathways_anatomy.json') as f:
    pathways_anatomy_data = json.load(f)  # contains the synapse counts/distributions for each m_type->m_type connection
with open(network_data_dir + 'pathways_physiology.json') as f:
    pathways_physiology_data = json.load(f)  # contains the synapse properties for each m_type->m_type connection

# lock x-dimension None if xy cross-section is square, otherwise lock x-dim to the specified length
x_dim_length = 200  

# scale down factor for the network
# number of neurons is scaled down by this factor,
# neuron density, layer thickness, and connection probability are held constant
scale_down_factor = 5
for layer in circuit_data['No. of neurons per layer']:
    circuit_data['No. of neurons per layer'][layer] = int(circuit_data['No. of neurons per layer'][layer] / scale_down_factor)

# add to the circuit_data the location of the top edge of each layer
# order is L1, L23, L4, L5, L6
layer_thicknesses = circuit_data['Layer thicknesses']
layer_thicknesses['L23'] = layer_thicknesses['L2'] + layer_thicknesses['L3']
circuit_data['Neuron densities']['L23'] = (circuit_data['Neuron densities']['L2'] + circuit_data['Neuron densities']['L3']) / 2
circuit_data['Layer top edges'] = {'L1': 0}
layers = ['L1', 'L23', 'L4', 'L5', 'L6']
for i_l in range(1, len(layers)):
    prev_thickness = circuit_data['Layer top edges'][layers[i_l-1]] + layer_thicknesses[layers[i_l-1]]
    circuit_data['Layer top edges'][layers[i_l]] = prev_thickness

# add to each the layer-specific xy-area: (cell count / neuron density * 1000^3 )/ layer thickness
circuit_data['Layer areas'] = {}
circuit_data['x-length'] = {} # in microns
circuit_data['y-length'] = {} # in microns
for layer in layers:
    circuit_data['Layer areas'][layer] = (circuit_data['No. of neurons per layer'][layer] / circuit_data['Neuron densities'][layer] * 1000**3) / layer_thicknesses[layer]
    if x_dim_length is None:
        circuit_data['x-length'][layer] = np.sqrt(circuit_data['Layer areas'][layer])
        circuit_data['y-length'][layer] = circuit_data['x-length'][layer]
    else:
        circuit_data['x-length'][layer] = x_dim_length
        circuit_data['y-length'][layer] = circuit_data['Layer areas'][layer] / x_dim_length
    circuit_data['x-length'][layer] = np.sqrt(circuit_data['Layer areas'][layer])
print([layer + ": " + str(circuit_data['x-length'][layer])[:5] + " x "  + 
       str(circuit_data['y-length'][layer])[:5] + 
       ' um' for layer in layers])

['L1: 169.1 x 169.1 um', 'L23: 155.3 x 155.3 um', 'L4: 166.2 x 166.2 um', 'L5: 166.5 x 166.5 um', 'L6: 165.7 x 165.7 um']


In [52]:
nml_doc = component_factory("NeuroMLDocument", id="Cortex_Network")

### Create the network
net = nml_doc.add("Network", id="Cortex_Network", validate=False)
net.type="networkWithTemperature" 
net.temperature="34.0degC"

neuron_population_dict = {}  # neuron_population_dict[m_type][e_type] = population_object
microcircuit_volume = circuit_data["Microcircuit volume"]
microcircuit_thickness = sum(circuit_data["Layer thicknesses"].values())

for layer in layers_data:
    layer_dict = layers_data[layer]
    e_type_counts = layer_dict['No. of neurons per electrical types']
    m_type_counts = layer_dict['No. of neurons per morphological types']

    # layer thickness
    layer_thickness = circuit_data["Layer thicknesses"][layer]
    layer_neuron_density = circuit_data["Neuron densities"][layer]
    layer_neuron_count = circuit_data["No. of neurons per layer"][layer]

    for m_type in m_type_counts:
        m_type_cells = [f for f in cell_files if m_type in f]
        m_count = m_type_counts[m_type]

        # for the e-types that match this m-type, build a relative probability distribution of these e-types based on their counts
        me_type_probs = {}
        e_count_sum = 0

        for e_type in e_type_counts:
            me_type_matches = [f for f in m_type_cells if e_type in f]
            for match in me_type_matches:
                me_type_probs[match] = e_type_counts[e_type] / len(me_type_matches)
            if len(me_type_matches) > 0:
                e_count_sum += e_type_counts[e_type]
        
        # normalize the probabilities
        for e_type in me_type_probs:
            me_type_probs[e_type] /= e_count_sum
        #print(me_type_probs)

        # probabilities should sum to 1
        if abs(1 - sum(me_type_probs.values())) > 0.0001 and len(me_type_probs.keys()) > 0:
            print("Error: probabilities do not sum to 1")
            print(sum(me_type_probs.values()))
            print(me_type_probs)

        # sample from the me-type distribution to get the number of cells of each type
        me_type_counts = {}  # maps me-type to # neurons of that type
        for i_morp in range(m_count):
            m_type_cells = list(me_type_probs.keys())
            m_type_probs = list(me_type_probs.values())
            if len(m_type_cells) == 0 or len(m_type_probs) == 0:
                continue
            me_type_sample = random.choices(m_type_cells, m_type_probs)[0]
            if me_type_sample not in me_type_counts:
                me_type_counts[me_type_sample] = 1
            else:
                me_type_counts[me_type_sample] += 1

        # create the neurons and place them in the network for this layer. Record in neuron_population_dict
        for me_type_sample in me_type_counts:
            print("Creating " + str(me_type_counts[me_type_sample]) + " cells of type " + me_type_sample)
            # read in the cell file
            cell_model = read_neuroml2_file(cell_data_dir + me_type_sample)

            # create the neuron population object and place them in the network for this layer
            size_pop1 = me_type_counts[me_type_sample]
            nml_doc.add("IncludeType", href=cell_data_dir + me_type_sample)

            population_obj = component_factory("Population", id="Exc", component=cell_model.id, size=size_pop1, type="populationList")
            
            # Set optional color property. Note: used later when generating plots
            ##pop0.add("Property", tag="color", value="0 0 .8")
            
            # set the x,y,z location of the population, based on layer thickness (z), model x,y dimensions, and cell count of this population
            # place the population at a random x,y location within the layer,
            #  uniformly distributed according to layer_thickness, layer_neuron_density, and layer_neuron_count
            xy_avail_area = circuit_data['Layer areas'][layer]
            x_y_length = np.sqrt(xy_avail_area)
            z_start_bound = circuit_data['Layer top edges'][layer]
            z_end_bound = z_start_bound + layer_thickness
            for i_cell in range(size_pop1):
                x = random.uniform(0, x_y_length)
                y = random.uniform(0, x_y_length)
                z = random.uniform(z_start_bound, z_end_bound)
                population_obj.add("Instance", id=str(i_cell), location=component_factory("Location", x=x, y=y, z=z))

            # add to neuron_population_dict
            if m_type not in neuron_population_dict:
                neuron_population_dict[m_type] = {}
            neuron_population_dict[m_type][me_type_sample] = population_obj


pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cACint209_L6_MC_5597e4d536_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:23:26,299 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cACint209_L6_MC_5597e4d536_0_0.cell.nml


Creating 20 cells of type cACint209_L6_MC_5597e4d536_0_0.cell.nml


pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cNAC187_L6_MC_8f46a0ad62_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:23:26,557 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cNAC187_L6_MC_8f46a0ad62_0_0.cell.nml


Creating 11 cells of type cNAC187_L6_MC_8f46a0ad62_0_0.cell.nml


pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L6_MC_d379cda689_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:23:26,833 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L6_MC_d379cda689_0_0.cell.nml
pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bIR215_L6_MC_6c575906a7_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:23:26,968 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bIR215_L6_MC_6c575906a7_0_0.cell.nml


...

Creating 1 cells of type cIR216_L1_HAC_5c847ea2ef_0_0.cell.nml
Creating 10 cells of type cNAC187_L1_SLAC_16735ed026_0_0.cell.nml
Creating 4 cells of type cNAC187_L1_SLAC_26e7460577_0_0.cell.nml


pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cNAC187_L1_SLAC_9d4b6f19da_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:26:05,242 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cNAC187_L1_SLAC_9d4b6f19da_0_0.cell.nml
pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cNAC187_L1_SLAC_1b78a7b294_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:26:05,318 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cNAC187_L1_SLAC_1b78a7b294_0_0.cell.nml
pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cNAC187_L1_SLAC_3759bf3d94_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:26:05,360 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/cNAC187_L1_SLAC_3759bf3d94_0_0.cell.nml
pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L1_SLAC_8a5944f8ac_0_0.cell.nml
pyNeuroML >>> 2025-

Creating 5 cells of type cNAC187_L1_SLAC_9d4b6f19da_0_0.cell.nml
Creating 4 cells of type cNAC187_L1_SLAC_1b78a7b294_0_0.cell.nml
Creating 10 cells of type cNAC187_L1_SLAC_3759bf3d94_0_0.cell.nml
Creating 1 cells of type bNAC219_L1_SLAC_8a5944f8ac_0_0.cell.nml
Creating 1 cells of type cACint209_L1_SLAC_0a908934ff_0_0.cell.nml


pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L1_SLAC_bb26216f9f_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:26:05,468 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L1_SLAC_bb26216f9f_0_0.cell.nml
pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L1_SLAC_9f043bb71a_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:26:05,535 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L1_SLAC_9f043bb71a_0_0.cell.nml
pyNeuroML >>> pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L1_SLAC_1aa0a09b13_0_0.cell.nml
pyNeuroML >>> 2025-01-31 13:26:05,593 - pyneuroml.pynml - INFO - Loading NeuroML2 file: ../NMC_model/NMC.NeuronML2/bNAC219_L1_SLAC_1aa0a09b13_0_0.cell.nml


Creating 3 cells of type bNAC219_L1_SLAC_bb26216f9f_0_0.cell.nml
Creating 2 cells of type bNAC219_L1_SLAC_9f043bb71a_0_0.cell.nml
Creating 1 cells of type bNAC219_L1_SLAC_1aa0a09b13_0_0.cell.nml


In [23]:
total_connections = 0
for m1m2_key in pathways_anatomy_data:
    syn_anatomy = pathways_anatomy_data[m1m2_key]
    syn_physiology = pathways_physiology_data[m1m2_key]

    m1, m2 = m1m2_key.split(":")
    connection_prob = syn_anatomy["connection_probability"]
    decay_mean = syn_physiology["decay_mean"]

    # simple mono-exponential synapse
    # <Property name="weight" dimension="none" defaultValue="1"/>

    # <Parameter name="tauDecay" dimension="time" description="Time course of decay"/>
    # <Parameter name="gbase" dimension="conductance" description="Baseline conductance, generally the maximum conductance following a single spike"/>
    # <Parameter name="erev" dimension="voltage" description="Reversal potential of the synapse"/>

    syn0 = nml_doc.add(
        "ExpOneSynapse", id=m1m2_key.replace(":", "_"), gbase="65nS", erev="0mV", tau_decay= str(decay_mean) + "ms"
    )
    #print("me-types:")
    #print(neuron_population_dict[m1])
    #print(neuron_population_dict[m2])
    # for every pair of me-types, create a fraction of the synapses
    for me1 in neuron_population_dict[m1]:
        me_pop1 = neuron_population_dict[m1][me1]
        for me2 in neuron_population_dict[m2]:
            me_pop2 = neuron_population_dict[m2][me2]

            # Create connections and inputs
            proj_count = 0

            projection = Projection(
                                id="Proj_",
                                presynaptic_population=me_pop1.id,
                                postsynaptic_population=me_pop2.id,
                                synapse=syn0.id,
                            )

            net.projections.append(projection)
            for i in range(me_pop1.size):
                for j in range(me_pop2.size):
                    if random.random() <= connection_prob: # probablistic connection...
                        connection = ConnectionWD(
                            id=proj_count,
                            pre_cell_id="%s[%i]" % (me_pop1.id, i),
                            post_cell_id="%s[%i]" % (me_pop2.id, j),
                            weight=random.random(),
                            delay='0ms'
                        )
                        projection.add(connection)
                        proj_count += 1
            print("\tAdded %i connections from %s to %s" % (proj_count, me1, me2))
            total_connections += proj_count
print("Total so far: %i" % total_connections + "\n\n")



	Added 135 connections from cADpyr231_L6_BPC_2f96fbbd13_0_0.cell.nml to cADpyr230_L4_SS_1afeb14f17_0_0.cell.nml
	Added 120 connections from cADpyr231_L6_BPC_2f96fbbd13_0_0.cell.nml to cADpyr230_L4_SS_08eefff078_0_0.cell.nml
	Added 145 connections from cADpyr231_L6_BPC_2f96fbbd13_0_0.cell.nml to cADpyr230_L4_SS_9e49de205b_0_0.cell.nml
	Added 143 connections from cADpyr231_L6_BPC_2f96fbbd13_0_0.cell.nml to cADpyr230_L4_SS_a8ef0f9d97_0_0.cell.nml
	Added 121 connections from cADpyr231_L6_BPC_2f96fbbd13_0_0.cell.nml to cADpyr230_L4_SS_a5ca6392b3_0_0.cell.nml
	Added 106 connections from cADpyr231_L6_BPC_3406e7f0b2_0_0.cell.nml to cADpyr230_L4_SS_1afeb14f17_0_0.cell.nml
	Added 82 connections from cADpyr231_L6_BPC_3406e7f0b2_0_0.cell.nml to cADpyr230_L4_SS_08eefff078_0_0.cell.nml
	Added 98 connections from cADpyr231_L6_BPC_3406e7f0b2_0_0.cell.nml to cADpyr230_L4_SS_9e49de205b_0_0.cell.nml
	Added 89 connections from cADpyr231_L6_BPC_3406e7f0b2_0_0.cell.nml to cADpyr230_L4_SS_a8ef0f9d97_0_0.cell

KeyboardInterrupt: 

In [18]:
# stimulus: L4 spiny stellate neurons (simulating thalamic input) as a pulse generator at 20ms, duration 0.2ms, amplitude on 10^-1 nA scale
for me_pop_key in neuron_population_dict["L4_SS"]:
    me_pop = neuron_population_dict["L4_SS"][me_pop_key]
    for i in range(0, me_pop.size):
        # pulse generator as explicit stimulus
        pg = nml_doc.add(
            "PulseGenerator",
            id="pg_exc_%i" % i,
            delay="20ms",
            duration="0.2ms",
            amplitude="%f nA" % (0.3 + 0.1 * random.random()),
        )
        
        exp_input = net.add(
            "ExplicitInput", target="%s[%i]" % (me_pop.id, i), input=pg.id
        )

print(nml_doc.summary())

nml_net_file = 'Cortex_Network.net.nml'
writers.NeuroMLWriter.write(nml_doc, nml_net_file)

print("Written network file to: " + nml_net_file)


pyNeuroML >>> 2025-01-29 16:19:41,711 - neuroml.nml.generatedssupersuper - WARNING - Explicit Input of type pg_exc_0 to Exc(cell 0), destination: unspecified already exists in explicit_inputs. Use `force=True` to force readdition.
pyNeuroML >>> 2025-01-29 16:19:41,713 - neuroml.nml.generatedssupersuper - WARNING - Explicit Input of type pg_exc_1 to Exc(cell 1), destination: unspecified already exists in explicit_inputs. Use `force=True` to force readdition.
pyNeuroML >>> 2025-01-29 16:19:41,714 - neuroml.nml.generatedssupersuper - WARNING - Explicit Input of type pg_exc_2 to Exc(cell 2), destination: unspecified already exists in explicit_inputs. Use `force=True` to force readdition.
pyNeuroML >>> 2025-01-29 16:19:41,716 - neuroml.nml.generatedssupersuper - WARNING - Explicit Input of type pg_exc_3 to Exc(cell 3), destination: unspecified already exists in explicit_inputs. Use `force=True` to force readdition.
pyNeuroML >>> 2025-01-29 16:19:41,718 - neuroml.nml.generatedssupersuper - W

*******************************************************
* NeuroMLDocument: Cortex_Network
*
*  ExpOneSynapse: ['L23_LBC_L23_NGC', 'L23_MC_L1_SLAC', 'L4_MC_L23_BTC', 'L4_PC_L6_TPC_L1', 'L5_LBC_L4_SS', 'L6_BP_L4_SS']
*  IncludeType: ['../NMC_model/NMC.NeuronML2/bAC217_L23_BP_2004171a4c_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_BTC_65225ca4f8_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_BTC_e5d9eb2ec3_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_DBC_393cc5cca5_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_DBC_3f985b6ce2_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_DBC_55c1d16c8c_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_DBC_e5ab405051_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_LBC_361cffb4c6_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_LBC_4738b5c12c_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_LBC_72e65525d3_0_0.cell.nml', '../NMC_model/NMC.NeuronML2/bAC217_L23_LBC_c5ee2b20c2_0_0.cell.nml', '../NMC_mo

pyNeuroML >>> pyneuroml.pynml - INFO - Executing: (java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "Cortex_Network.net.nml" ) in directory: .
pyNeuroML >>> 2025-01-29 16:19:44,203 - pyneuroml.pynml - INFO - Executing: (java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "Cortex_Network.net.nml" ) in directory: .


Written network file to: Cortex_Network.net.nml


pyNeuroML >>> pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "Cortex_Network.net.nml" ' returned non-zero exit status 1.
pyNeuroML >>> 2025-01-29 16:19:46,627 - pyneuroml.pynml - CRITICAL - *** Problem running command: 
       Command 'java -Xmx400M  -jar  "C:\Users\jjudge3\AppData\Local\anaconda3\envs\in-silico-hVOS\Lib\site-packages\pyneuroml\lib\jNeuroML-0.11.0-jar-with-dependencies.jar" -validate "Cortex_Network.net.nml" ' returned non-zero exit status 1.
pyNeuroML >>> pyneuroml.pynml - CRITICAL -  jNeuroML >>   jNeuroML v0.11.0
 jNeuroML >>  Validating: c:\Users\jjudge3\Desktop\scratch\in-silico-hVOS\Cortex_Network.net.nml
 jNeuroML >>  
pyNeuroML >>> 2025-01-29 16:19:46,629 - pyneuroml.pynml - CRITICAL -  jNeuroML >>   jNeuroML v0.11.0
 jNeuroML >>  Validating: c:\Users\jjudge3\Desktop\s

False

In [ ]:
# next, generate a LEMS file to run the simulation, with helper method
# https://docs.neuroml.org/Userdocs/LEMSSimulation.html
# https://github.com/NeuroML/pyNeuroML/blob/master/pyneuroml/lems/__init__.py#L19 

sim_id = "CortexNet"
target = "simplenet"
duration = 200
dt = 0.025
lems_file_name = "LEMS_%s.xml" % sim_id
target_dir = "test_data"


# TO DO: to save computation time, only generate/save for cells within a 200-um slice of the network.
generate_lems_file_for_neuroml(
    sim_id,
    nml_net_file,
    target,
    duration,
    dt,
    lems_file_name,
    target_dir,
    include_extra_files=[],
    gen_plots_for_all_v=True,
    plot_all_segments=True,
    gen_plots_for_quantities={},  # Dict with displays vs lists of quantity paths
    gen_plots_for_only_populations=[],  # List of populations, all pops if = []
    gen_saves_for_all_v=True,
    save_all_segments=True,
    gen_saves_for_only_populations=[],  # List of populations, all pops if = []
    gen_saves_for_quantities={},  # Dict with file names vs lists of quantity paths
    gen_spike_saves_for_all_somas=True,
    report_file_name="report.txt",
    copy_neuroml=True,
    verbose=True,
)